#### **Spatial Integration**

**1. By hand**

* How many integration points are required to get an exact integral for a polynomial of order 6 (highest power of x is 6)
    * using Newton-Cotes integration?
    * using Gauss-Legendre integration?
* Why does Gauss-Legendre integration generally require fewer integration points than Newton-Cotes for the same accuracy?
* Derive the weights for 3-point Newton-Cotes integration. Also, list the corresponding coordinates of the integration points**

**2 Gauss-Legendre:   Write Python functions to perform integration using the following methods.**

* Newton-Cotes (for 2-point to 4-point integration)
* Gauss-Legendre (use the code in the book to calculate the weights and locations of the integration points or hardwire the values for 2-point to 5-point integration)

Obtain the weights and integration points from the book, web, or a Python package. Validate your functions by performing integrations for polynomial integrands. If you choose an appropriate number of integration points, you should get the exact solution.

In [36]:
import numpy as np
from math import sqrt

#NewtonCotes from online source
def newtonCotes(f, a, b, n):
    h = (b-a) / (n-1)

    #weights and such
    #as you can see, the more points, the more wacky
    if n == 2:
        return .5 * h * (f(a) + f(a + h))
    if n == 3:
        return (1/3) * h * (f(a) + 4*f(a + h) + f(a + 2*h))
    if n == 4:
        return (3/8) * h * (f(a) + 3*f(a + h) + 3 * f(a + 2*h) + f(a + 3*h))
    if n == 5:
        return (2/45) * h * (7*f(a) + 32*f(a + h) + 12 * f(a + 2*h) + 32*f(a + 3*h) + 7*f(a + 4*h)) 

#Gauss-Legendre
def gaussQuadrature(f,a,b,n):
    [rho, w] = np.polynomial.legendre.leggauss(n)

    x = .5*(1 - rho)*a + .5*(1 + rho)*b 
    J = (b - a)/2

    I_GL = 0 #summation initialization

    for i in range(len(rho)):
        I_GL += w[i]*f(x[i])

    I_GL *= J
        
    return I_GL

**3. Evaluate $\int_a^b {{{\left( {30 - {x^2}} \right)}^{3/2}}dx}$ by hand with each of these methods:**
* Use 3-point Newton-Cotes integration for a = 1, b= 5
* Use 2-point Gauss-Legendre integration for a = 1, b= 5


**4. Use the Python functions you wrote earlier to evaluate $\int_a^b {{{\left( {30 - {x^2}} \right)}^{3/2}}dx}$.**

* Use 5-point Newton-Cotes integration for a = 1, b= 5
* Use 3-point Gauss-Legendre integration for a = 1, b= 5


In [37]:
def F2(x):
    return ((30 - x*x)**(3/2)) # Function to be integrated

a = 1.
b = 5.

#Newton-Cotes
I1 = newtonCotes(F2, a, b, 5)
print(f"The solution using {5} point Newton-Cotes is {I1}.")

#Gauss-Legendre
I2 = gaussQuadrature(F2, a, b, 3)
print(f"The solution using {3} point Gauss-Legendre is {I2}.")

The solution using 5 point Newton-Cotes is 366.44029576771516.
The solution using 3 point Gauss-Legendre is 366.32065086137953.


**5. Problem Set 6.2, p. 230: Problem 1**

In [38]:
def F2(x):
    return (np.log(x))/(x**2 -2*x + 2) # Function to be integrated

#Gauss-Legendre
a = 1.
b = np.pi

nvals = [2, 4]

for n in nvals:
    I = gaussQuadrature(F2, a, b, n)
    print(f"The solution using {n} point Gauss-Legendre is {I}")

The solution using 2 point Gauss-Legendre is 0.6067250228624489
The solution using 4 point Gauss-Legendre is 0.5847680362127091


**6. Problem Set 6.2 on p. 231: Problem 14.** Use 5-point Gauss-Legendre integration.

In [39]:
def F2(x, c):
    return x**2 * np.sqrt(1 + (2 * c * x)**2) # just add a c value you can change

#Gauss-Legendre
a = 0.
b = 1

cvals = [.5, 1, 2]

#Gauss-Legendre
def gaussQuadratureC(f,a,b,n,c):
    [rho, w] = np.polynomial.legendre.leggauss(n)

    x = .5*(1 - rho)*a + .5*(1 + rho)*b 
    J = (b - a)/2

    I_GL = 0 #summation initialization

    for i in range(len(rho)):
        I_GL += w[i]*f(x[i], c)

    I_GL *= J
        
    return I_GL

for c in cvals:
    I = gaussQuadratureC(F2, a, b, 5, c)
    print(f"The solution using {5} point Gauss-Legendre for c = {c} is {I}.")

The solution using 5 point Gauss-Legendre for c = 0.5 is 0.4201583764271097.
The solution using 5 point Gauss-Legendre for c = 1 is 0.6063378036262291.
The solution using 5 point Gauss-Legendre for c = 2 is 1.0588953460037933.


**7. Problem Set 6.2, p. 230: Problem 16.** Clarification: First fit a cubic polynomial to the pressure data using least squares. Then perform numerical integration of the approximate integrands.

In [40]:
#Gauss-Legendre
def gaussQuadrature(f,a,b,n):
    [rho, w] = np.polynomial.legendre.leggauss(n)

    x = .5*(1 - rho)*a + .5*(1 + rho)*b 
    J = (b - a)/2

    I_GL = 0 #summation initialization

    for i in range(len(rho)):
        I_GL += w[i]*f(x[i])

    I_GL *= J
        
    return I_GL

#combined least squares and plotting
def LeastSquares(x, y, m):

    # Create H
    n = len(x)
    H = np.zeros((n, m+1)) #5 x 2, where 2 is the degree + 1

    for j in range(m+1):
        for i in range(n):
            H[i, j]= x[i]**j #actually fills in H matrix

    HTH = H.T @ H
    HTy = H.T @ y
    
    coefficients = np.linalg.solve(HTH, HTy)

    return coefficients

x = np.array([0, 15, 35, 52, 80, 112])
y = np.array([310, 425, 530, 575, 612, 620])

coefficients = LeastSquares(x, y, 3)

print(f"Least Squares Polynomial: {coefficients[0]} + {coefficients[1]}*x + {coefficients[2]}*x**2 + {coefficients[3]}*x**3")

def F2(x):
    return coefficients[0] + coefficients[1] * x + coefficients[2] * x**2 + coefficients[3] * x**3 

def NumeratorF(x):
    return x * F2(x)

I1 = gaussQuadrature(F2, x[0], x[-1], 2) #
I2 = gaussQuadrature(NumeratorF, x[0], x[-1], 3) #function is the NumeratorF

ratio = I2/I1

print(f"The height of the pressure center is {ratio} m.")

Least Squares Polynomial: 309.98028156864297 + 8.988198144924961*x + -0.08981721771636532*x**2 + 0.00030620326915038396*x**3
The height of the pressure center is 60.57303049810242 m.
